## Load data:



In [ ]:
df <- read.csv2("data/data.csv", sep = ";")
df


## Preprocess data:



In [ ]:
library("tidyverse")
library("lubridate")
library("prophet")

prophet_df <- select(df, YEAR, MONTH, DAY, CNT)
prophet_df <- prophet_df %>% mutate(ds = make_date(YEAR, MONTH, DAY))
prophet_df <- select(prophet_df, ds, CNT)
colnames(prophet_df) <- c("ds", "y")
prophet_df$y <- log(prophet_df$y)


## Split into train and test sets:



In [ ]:
train <- head(prophet_df, 586)
sid <- as.numeric(rownames(train))
test <- prophet_df[-sid,]


## Train Prophet model:



In [ ]:
m <- prophet(train, daily.seasonality = FALSE, yearly.seasonality = TRUE)
future <- make_future_dataframe(m, periods = 146, include_history = FALSE)
forecast <- predict(m, future)
forecast[c("ds", "yhat")]


## Calculate awfully large MAPE:



In [ ]:
library("MLmetrics")
MAPE(y_pred = exp(forecast$yhat), y_true = exp(test$y))


## Plot forecast:



In [ ]:
plot(m, forecast)
prophet_plot_components(m, forecast)
